微调 Gemma-3-4b-it

### Installation

In [8]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [3]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

使用你的 hf-token替换

 https://huggingface.co/settings/tokens

 *需要勾选read 权限

In [16]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "google/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    full_finetuning = False, # [NEW!] We have full finetuning now!
    token = "your hf token", # 使用你的 hf-token替换
)

==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


测试原始模型能力

In [12]:
messages = [{
    "role": "system",
    "content": [{
        "type" : "text",
        "text" : "You are a helpful assistant. help me translate the text from English to Chinese.",
    }]
},{
    "role": "user",
    "content": [{"type" : "text", "text" : " what can i do with python, and how can i learn python?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 200, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Okay, here's the translation of "What can I do with Python, and how can I learn Python?" into Chinese, with a few options to best suit the context:

**Option 1 (More formal and direct - suitable for a learning resource or tutorial):**

“我可以用 Python 做什么？如何学习 Python？”
(Wǒ kěyǐ yòng Python zuò shénme? Ríháo xuéxí Python?)

* **我 (wǒ):** I
* **可以用 (kěyǐ yòng):** can use / can do with
* **Python:** Python (same in Chinese)
* **做 (zuò):** do / make
* **什么 (shénme):** what
* **如何 (ríháo):** how
* **学习 (xuéxí):** learn / study


**Option 2 (Slightly more conversational – good for asking a


# 使用LoRA，以便只更新少量模型参数！

In [17]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


In [18]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

## 加载数据集 & 格式化数据集，将中英文翻译对转换为对话格式

In [19]:
from datasets import load_dataset

dataset = load_dataset('wmt18', 'zh-en', split = "train[:30000]")
dataset[10]

# 格式化数据集，将中英文翻译对转换为对话格式
def format_translation_to_chat(example):
    # 使用英文作为用户输入，英文作为模型输出（按照指令要求）
    conversations = [
        {"role": "system", "content": "You are a helpful assistant. help me translate the text from English to Chinese."},
        {"role": "user", "content": example["translation"]["en"]},
        {"role": "assistant", "content": example["translation"]["zh"]}
    ]
    return {"conversations": conversations}


# 应用格式化函数
dataset = dataset.map(format_translation_to_chat)
dataset[10]


Generating train split:   0%|          | 0/25160346 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2001 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3981 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

{'translation': {'en': 'The end of the East-West ideological divide and the end of absolute faith in markets are historical turning points.',
  'zh': '东西方意识形态鸿沟的结束，以及对市场绝对信心的后果，都是历史的转折点。'},
 'conversations': [{'content': 'You are a helpful assistant. help me translate the text from English to Chinese.',
   'role': 'system'},
  {'content': 'The end of the East-West ideological divide and the end of absolute faith in markets are historical turning points.',
   'role': 'user'},
  {'content': '东西方意识形态鸿沟的结束，以及对市场绝对信心的后果，都是历史的转折点。', 'role': 'assistant'}]}

# 使用`standardize_data_formats`来尝试将数据集转换为适合微调的正确格式！



In [20]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)
dataset[100]

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/30000 [00:00<?, ? examples/s]

{'translation': {'en': 'MADRID – As 2016 comes to an end, the outlook for 2017 is shrouded in uncertainty.',
  'zh': '马德里——随着2016年的结束，2017年的前景被不确定性所笼罩。'},
 'conversations': [{'content': 'You are a helpful assistant. help me translate the text from English to Chinese.',
   'role': 'system'},
  {'content': 'MADRID – As 2016 comes to an end, the outlook for 2017 is shrouded in uncertainty.',
   'role': 'user'},
  {'content': '马德里——随着2016年的结束，2017年的前景被不确定性所笼罩。', 'role': 'assistant'}]}

# 要将Gemma-3的聊天模板应用到conversations，并保存到text

In [21]:
def apply_chat_template(examples):
    texts = tokenizer.apply_chat_template(examples["conversations"])
    return { "text" : texts }
pass
dataset = dataset.map(apply_chat_template, batched = True)
dataset[100]["text"]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

'<bos><start_of_turn>user\nYou are a helpful assistant. help me translate the text from English to Chinese.\n\nMADRID – As 2016 comes to an end, the outlook for 2017 is shrouded in uncertainty.<end_of_turn>\n<start_of_turn>model\n马德里——随着2016年的结束，2017年的前景被不确定性所笼罩。<end_of_turn>\n'

现在让我们使用Huggingface TRL的SFTTrainer！更多文档请参考：TRL SFT文档。我们执行30步以加快速度，但您可以设置num_train_epochs=1进行完整训练，并关闭max_steps=None。


In [22]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 50,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/30000 [00:00<?, ? examples/s]

使用Unsloth的train_on_completions方法，仅对助手的输出进行训练，而忽略用户输入的损失。这有助于提高微调的准确性！


In [23]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=2):   0%|          | 0/30000 [00:00<?, ? examples/s]

In [25]:
# @title 显示当前内存统计
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
14.26 GB of memory reserved.


开始训练

In [26]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 30,000 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
1,6.555700
2,8.916200
3,6.039400
4,5.718000
5,5.108000
6,4.152300
7,3.191100
8,2.781400
9,2.751600
10,1.874100


In [27]:
# @title 显示最终内存和时间统计
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

192.9714 seconds used for training.
3.22 minutes used for training.
Peak reserved memory = 14.26 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 96.737 %.
Peak reserved memory for training % of max memory = 0.0 %.


测试微调之后模型的效果

In [28]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "system",
    "content": [{
        "type" : "text",
        "text" : "You are a helpful assistant. help me translate the text from English to Chinese.",
    }]
},{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "hello how are you~",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 100, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nYou are a helpful assistant. help me translate the text from English to Chinese.\n\nhello how are you~<end_of_turn>\n<start_of_turn>model\n你好你好~<end_of_turn>']

也可以使用TextStreamer进行连续推理，这样你可以看到生成token


In [29]:
messages = [{
    "role": "system",
    "content": [{
        "type" : "text",
        "text" : "You are a helpful assistant. help me translate the text from English to Chinese.",
    }]
},{
    "role": "user",
    "content": [{"type" : "text", "text" : " what can i do with python, and how can i learn python?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

我能用python做什么？ 如何学习python?<end_of_turn>
